In [1]:
import numpy as np
import rasterio
import os

from rasterio.enums import Resampling
from pathlib import Path

import sys
if sys.version_info < (3, 9):
    from importlib_resources import files
else:
    from importlib.resources import files

from beak.utilities.io import save_raster
from beak.utilities.raster_processing import unify_raster_grids

DATA_PATH = files('beak.data')

# Tests
Usually, unifying incorporates several steps. <br>
Manually, these steps are:

1. Reproject
2. Resample
3. Snap
4. Clip

However, it might be, that this order needs to be changed in some cases. <br>
The provided tool is also capable of clipping the cells based on the <code>nodata</code> locations of the base raster.<p>
**Note**: If the base raster contains <code>nodata</code> values within the area of interest, theses cells will be clipped, too. <br>

## Case 1
**Grids to dataframe export with U.S. continental states extent**<p>
Conditions:
- Same CRS
- Different cell sizes
- Different number of cells
- Different extent

In [2]:
RAW_PATH = DATA_PATH / "MCCAFFERTY23-RAW" / "Geophysics"
TEST_FILE = RAW_PATH / "GeophysicsMagRTP.tif"

base_raster = DATA_PATH / "BASE_RASTERS" / "EPSG_4326_RES_0_025_CLIPPED_USC.tif"
test_rasters = [TEST_FILE]
out_path = "CASE_1_" + os.path.basename(TEST_FILE)

# Base raster is the first tuple in the list, followed by the unified rasters
unified_raster, meta = unify_raster_grids(base_raster, test_rasters, resampling_method=Resampling.bilinear, same_extent=True)[0]  


In [ ]:
save_raster(out_path, 
            array=unified_raster, 
            crs=meta['crs'], 
            height=meta['height'], 
            width=meta['width'], 
            nodata_value=meta['nodata'], 
            transform=meta['transform'],
            dtype="float32")

print("Unified raster saved to: ", out_path)

## Case 2
**Grid to grid from large extent low resolution to smaller extent high resolution with clipping to smaller extent**<p>

Conditions:
- Different CRS
- Different cell sizes
- Different number of cells
- Different extent
- Masking of cells based on <code>nodata</code> locations of the base raster

In [ ]:
RAW_PATH = DATA_PATH / "MCCAFFERTY23-RAW" / "Geophysics"
TEST_FILE = RAW_PATH / "GeophysicsMagRTP.tif"

base_raster = rasterio.open(DATA_PATH / "AEROMAGNETICS-RAW" / "SMidCont_RTP_UTM15.tif")
test_rasters = [rasterio.open(TEST_FILE)]
out_path = "CASE_2_" + os.path.basename(TEST_FILE)  

# Base raster is the first tuple in the list, followed by the unified rasters
unified_raster, meta = unify_raster_grids(base_raster, test_rasters, resampling_method=Resampling.bilinear, same_extent=True, same_shape=True)

save_raster(out_path, 
            array=unified_raster, 
            crs=meta['crs'], 
            height=meta['height'], 
            width=meta['width'], 
            nodata_value=meta['nodata'], 
            transform=meta['transform'],
            dtype="float32")

print("Unified raster saved to: ", out_path)